In [47]:
from langchain.chat_models import AzureChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool, QuerySQLCheckerTool
import pandas as pd

In [48]:
azure_llm = AzureChatOpenAI(
    openai_api_base = 'https://openai-lh.openai.azure.com/openai/',
    openai_api_version = '2023-06-01-preview',
    deployment_name = 'LH-GPT',
    openai_api_key = '312ff50d6d954023b8748232617327b6',
    temperature = 0
) 

c:\Users\soumyadipghorai\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_community\chat_models\azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
c:\Users\soumyadipghorai\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_community\chat_models\azure_openai.py:182: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azure.openai.com/openai/deployments/example-deployment. Updating https://openai-lh.openai.azure.com/openai/ to https://openai-lh.openai.azure.com/openai/.
  warnings.warn(


In [ ]:
azure_llm.predict()

In [49]:
db = SQLDatabase.from_uri("sqlite:///database/sample_data.sqlite3")

In [50]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import ChatOpenAI

toolkit = SQLDatabaseToolkit(db=db, llm=azure_llm)
context = toolkit.get_context()

print(context)

{'table_info': '\nCREATE TABLE mock_data (\n\tcustomer_id INTEGER, \n\taccount_type TEXT, \n\tbalance REAL, \n\tincome REAL, \n\tcredit_score INTEGER, \n\tlast_transaction_date TEXT, \n\taverage_monthly_deposit REAL, \n\taverage_monthly_withdrawal REAL\n)\n\n/*\n3 rows from mock_data table:\ncustomer_id\taccount_type\tbalance\tincome\tcredit_score\tlast_transaction_date\taverage_monthly_deposit\taverage_monthly_withdrawal\n627\tinvestment\t917022.16\t877548.83\t750\t1/7/2018\t46042.1\t3422.32\n628\tchecking\t539950.14\t588337.85\t646\t1/8/2018\t67204.14\t75002.83\n629\tchecking\t286228.4\t303272.27\t316\t1/12/2018\t27404.68\t64359.12\n*/\n\n\nCREATE TABLE new_table (\n\tcompany_code INTEGER, \n\torder_line_number TEXT, \n\tpurchasing_order_date TEXT, \n\tmonth INTEGER, \n\tyear INTEGER, \n\tpo_description TEXT, \n\tmaterial_number TEXT, \n\tvendor_number REAL, \n\tplant_name INTEGER, \n\tpurchase_order_quantity REAL, \n\tunit_of_measure TEXT, \n\tunit_price REAL, \n\ttotal_order_value 

In [57]:
def execute_sql_query(feature_parameters: dict, user_query: str) -> str :  
    all_features = feature_parameters["feature"]

    print(all_features)

    # question = f"""
    # Give me all the values of date, {all_features} from the database.
    # """

    toolkit = SQLDatabaseToolkit(db=db, llm=azure_llm)
    context = toolkit.get_context()

    question = f"""
    You are an intelligent AI agent to generate sql query based on the user query. \n
    Your task is to return all the values of date, {all_features} from the database \n
    and apply filter based on the filters given below :
    {feature_parameters["filter"]}.

    also consider the database context given below : 
    {context}

    NOTE: DON'T apply order or limit.
    """

    execute_query = QuerySQLDataBaseTool(db=db) 
    query_checker = QuerySQLCheckerTool(db=db, llm=azure_llm)
    write_query = create_sql_query_chain(azure_llm, db)

    print(context)
    chain = write_query | query_checker | execute_query
    # chain = write_query 
    
    result = chain.invoke({
        "question": question 
    }) 

    print(result)

    return result 

In [68]:
result = execute_sql_query({
    'feature' : 'total order value', 
    'filter' : [
        'l1 category', 'Packaging Material'
    ]
}, user_query = "predict the balance of next month based on investment type")

total order value
{'table_info': '\nCREATE TABLE mock_data (\n\tcustomer_id INTEGER, \n\taccount_type TEXT, \n\tbalance REAL, \n\tincome REAL, \n\tcredit_score INTEGER, \n\tlast_transaction_date TEXT, \n\taverage_monthly_deposit REAL, \n\taverage_monthly_withdrawal REAL\n)\n\n/*\n3 rows from mock_data table:\ncustomer_id\taccount_type\tbalance\tincome\tcredit_score\tlast_transaction_date\taverage_monthly_deposit\taverage_monthly_withdrawal\n627\tinvestment\t917022.16\t877548.83\t750\t1/7/2018\t46042.1\t3422.32\n628\tchecking\t539950.14\t588337.85\t646\t1/8/2018\t67204.14\t75002.83\n629\tchecking\t286228.4\t303272.27\t316\t1/12/2018\t27404.68\t64359.12\n*/\n\n\nCREATE TABLE new_table (\n\tcompany_code INTEGER, \n\torder_line_number TEXT, \n\tpurchasing_order_date TEXT, \n\tmonth INTEGER, \n\tyear INTEGER, \n\tpo_description TEXT, \n\tmaterial_number TEXT, \n\tvendor_number REAL, \n\tplant_name INTEGER, \n\tpurchase_order_quantity REAL, \n\tunit_of_measure TEXT, \n\tunit_price REAL, \n\t

In [69]:
pd.DataFrame(eval(result))

,0,1
0,5/13/2021,34246.5000
1,12/17/2021,27021.5000
2,12/17/2021,11892.6390
3,12/17/2021,22964.4800
4,5/22/2021,8906.2400
...,...,...
18228,9/8/2021,103278.2064
18229,5/22/2021,-74746.7884
18230,5/22/2021,-55688.6528
18231,7/29/2021,-91999.4344
